In [ ]:
!gdown --id'' --output train.npy
!gdown --id'' --output test.npy

import numpy as np

train = np.load('train.npy',allow_pickle=True)
test = np.load('test.npy',allow_pickle=True)


In [ ]:
#半监督 anomaly dection
#检测出测试集中没有的 label
task ='pca'

In [ ]:
#KNN
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import f1_score,pairwise_distances,roc_auc_score
from scipy.cluster.vq import vq,kmeans

if task =='knn':
    x = train.reshape(len(train),-1)
    y = train.reshape(len(test),-1)
    scores = list()
    for n in range(1,10):
        kmeans_x = MiniBatchKMeans(n_clusters=n,batch_size =100).fit(x)
        y_cluster = kmeans_x.predict(y)
        y_dist = np.sum(np.square(kmeans_x.cluster_centers_[y_cluster]-y),axis=1)

        y_pred = y_dist


In [ ]:
#PCA
from sklearn.decomposition import PCA

if task =='pca':
    x = train.reshape(len(train),-1)
    y = train.reshape(len(test),-1)
    pca =PCA(n_components=2).fit(x)

    y_projected = pca.transform(y)
    y_reconstructed = pca.inverse_transform(y_projected)
    dist = np.sqrt(np.sum(np.square(y_reconstructed-y).reshape(len(y),-1),axis=1))

    y_pred = dist


In [ ]:
#Autoencoder
import torch 
from torch import nn 
import torch.nn.functional as F

class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(32*32*3,128),
            nn.ReLU(True),
            nn.Linear(128,64),
            nn.ReLU(True),
            nn.Linear(64,12),
            nn.ReLU(True),
            nn.Linear(12,3)
        )
        self.decoder =nn.Sequential(
            nn.Linear(3,12),
            nn.ReLU(True),
            nn.Linear(12,64),
            nn.ReLU(True),
            nn.Linear(64,128),
            nn.ReLU(True),
            nn.Linear(128,3*32*32),
            nn.Tanh()
        )
    
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3,12,4,stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(12,24,4,stride=2,padding=1),
            nn.ReLU(),
            nn.Conv2d(24,48,4,stride=2,padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48,24,4,stride=2,padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(24,12,4,stride=2,padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(12,3,4,stride=2,padding=1),
            nn.ReLU(),
            nn.Tanh()
        )
    
    def forward(self,x):
        x =self.encoder(x)
        x = self.decoder(x)
        return x 
    
class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()
        self.fc1 = nn.Linear(32*32*3,400)
        self.fc21 = nn.Linear(400,20)
        self.fc22 = nn.Linear(400,20)
        self.fc3 = nn.Linear(20,400)
        self.fc4 = nn.Linear(400,32*32*3)
    
    def encode(self,x):
        h1 =F.relu(self.fc1(x))
        return self.fc21(h1),self.fc22(h1)
    
    def reparametrize(self,mu,logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self,z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self,x):
        mu,logvar = self.encode(x)
        z = self.reparametrize(mu,logvar)
        return self.decode(z),mu,logvar


def loss_vae(recon_x,x,mu,logvar,criterion):
    mse = criterion(recon_x,x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(0.5)
    return mse+KLD


In [ ]:
#training